In [1]:
!pip install Pyfhel
!pip install pynacl
!pip install cryptography
!pip install tqdm

In [2]:
no_clients = 3
epochs = 3

In [3]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

# Check if GPU is available
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    print("GPUs available:", len(gpus))
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU available.")

2024-06-17 21:25:03.954732: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 21:25:03.954764: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 21:25:03.954790: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-17 21:25:03.961626: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 21:25:04.715747: W tensorflow/compiler/

TensorFlow version: 2.14.1
GPUs available: 1
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-06-17 21:25:05.787453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-17 21:25:05.806000: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-17 21:25:05.806221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
import Pyfhel
import tensorflow as tf
from tqdm import tqdm
import copy
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle
import sys
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import os
import tensorflow as tf
from Pyfhel import Pyfhel

# from src.models.FMLEE import FMLEE
# from src.data.load_data import load_mnist

In [5]:
import os

os.environ["TF_USE_LEGACY_KERAS"] = "True"

In [6]:


save_dir = "dataset/mnist_data/"
os.makedirs(save_dir, exist_ok=True)

In [7]:
import tensorflow as tf


class MAML(tf.keras.Model):
    def __init__(self, model):
        super(MAML, self).__init__()
        self.model = model

    def call(self, inputs):
        x = tf.reshape(inputs, (-1, 28, 28, 1))  # Reshape the input tensor
        return self.model(x)

    def get_config(self):
        return {"model": self.model.get_config()}

    @classmethod
    def from_config(cls, config):
        model = tf.keras.models.Model.from_config(config["model"])
        return cls(model)

    def train_step(self, data):
        x, y = data
        x = tf.reshape(x, (-1, 28, 28, 1))  # Reshape the input tensor
        y = tf.reshape(y, (-1,))  # Reshape the target labels
        with tf.GradientTape() as tape:
            y_pred = self.model(x)
            loss = self.compiled_loss(y, y_pred)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data
        x = tf.reshape(x, (-1, 28, 28, 1))  # Reshape the input tensor
        y = tf.reshape(y, (-1,))  # Reshape the target labels
        y_pred = self.model(x)
        self.compiled_loss(y, y_pred)
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

In [8]:
class FMLEE:
    def __init__(self, no_clients, epochs):
        self.no_clients = no_clients
        self.epochs = epochs
        self.HE = self.CKKS()
        self.clients = []
        self.init_clients()

    def model_spec(self):
        model = tf.keras.models.Sequential(
            [
                tf.keras.layers.Conv2D(
                    32, (3, 3), activation="relu", input_shape=(28, 28, 1)
                ),
                tf.keras.layers.MaxPooling2D((2, 2)),
                tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
                tf.keras.layers.MaxPooling2D((2, 2)),
                tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(64, activation="relu"),
                tf.keras.layers.Dense(10),
            ]
        )
        return model

    def init_model(self):
        model = MAML(self.model_spec())
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"],
        )
        return model

    def CKKS(self):
        HE = Pyfhel()
        ckks_params = {
            "scheme": "CKKS",
            "n": 2**14,  # Polynomial modulus degree. For CKKS, n/2 values can be
            "scale": 2**30,  # All the encodings will use it for float->fixed point
            "qi_sizes": [
                60,
                30,
                30,
                30,
                60,
            ],
        }
        HE.contextGen(**ckks_params)  # Generate context for ckks scheme
        HE.keyGen()  # Key Generation: generates a pair of public/secret keys
        HE.rotateKeyGen()
        HE.relinKeyGen()
        return HE

    def init_clients(self):
        for i in range(self.no_clients):
            self.clients.append(self.init_model())
            print(f"Client {i} initialized.")

In [9]:
def download_and_save_mnist(save_dir):
    (x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Save training data with progress bar
    for array, name in zip(
        [x_train_all, y_train_all, x_test, y_test],
        ["x_train.npy", "y_train.npy", "x_test.npy", "y_test.npy"],
    ):
        with tqdm(total=len(array), desc=f"Saving {name}") as pbar:
            np.save(os.path.join(save_dir, name), array)
            pbar.update(len(array))

    print(f"Dataset downloaded and saved locally at {save_dir}")


def load_mnist_from_local(save_dir):
    x_train_all = np.load(os.path.join(save_dir, "x_train.npy"))
    y_train_all = np.load(os.path.join(save_dir, "y_train.npy"))
    x_test = np.load(os.path.join(save_dir, "x_test.npy"))
    y_test = np.load(os.path.join(save_dir, "y_test.npy"))
    print(f"Dataset loaded from local files at {save_dir}")
    x_train_all = x_train_all.astype(np.float32) / 255
    x_test = x_test.astype(np.float32) / 255

    return (x_train_all, y_train_all), (x_test, y_test)


def load_mnist():
    if not os.path.exists(os.path.join(save_dir, "x_train.npy")):
        download_and_save_mnist(save_dir)
    return load_mnist_from_local(save_dir)

In [10]:
fml = FMLEE(no_clients, epochs)

2024-06-17 21:25:06.734248: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-17 21:25:06.734506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-17 21:25:06.734677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

RuntimeError: Bad StatusOr access: INTERNAL: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_UNKNOWN: unknown error

In [ ]:
fml.clients

[<MAML name=maml, built=False>,
 <MAML name=maml_1, built=False>,
 <MAML name=maml_2, built=False>]

In [ ]:
fml.HE

<ckks Pyfhel obj at 0x71805ccf07b0, [pk:Y, sk:Y, rtk:Y, rlk:Y, contx(n=16384, t=0, sec=128, qi=[60, 30, 30, 30, 60], scale=1073741824.0, )]>

In [ ]:
(x_train_all, y_train_all), (x_test, y_test)  = load_mnist()


Dataset loaded from local files at dataset/mnist_data/


In [ ]:
fml.clients[0].fit(x_train_all, y_train_all)

/home/cs-lab-12/miniconda3/envs/f39/lib/python3.9/site-packages/keras/src/backend/tensorflow/trainer.py:603: UserWarning: `model.compiled_loss()` is deprecated. Instead, use `model.compute_loss(x, y, y_pred, sample_weight)`.
  warnings.warn(
/home/cs-lab-12/miniconda3/envs/f39/lib/python3.9/site-packages/keras/src/backend/tensorflow/trainer.py:578: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.8945 - loss: 0.5350
